<a href="https://colab.research.google.com/github/weedge/doraemon-nb/blob/main/Train_your_tokenizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

如果您在colab上打开这个Notebook，您可能需要安装🤗Transformers和🤗Datasets。取消下面单元格的注释并执行它:

In [1]:
!pip install -q datasets transformers[sentencepiece]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.4 MB/s eta 0:00:00


如果您在本地打开这个笔记本，请确保您的环境安装了最新版本的Datasets和原始版本的Transformers。

In [2]:
from transformers.utils import send_example_telemetry

send_example_telemetry("tokenizer_training_notebook", framework="none")

## Loading the dataset

# Training your own tokenizer from scratch

在这个笔记本中，我们将看到几种从头开始在给定语料库上训练自己的分词器的方法，这样你就可以用它来从头开始训练一个语言模型。

为什么需要*训练*一个分词器呢？因为 Transformer 模型很常用子词分词算法，并且需要根据你正在使用的语料库来训练，以识别单词中经常出现的部分。我们建议你参考 Hugging Face 课程中的[分词章节](https://huggingface.co/course/chapter2/4?fw=pt)来了解分词器的基本介绍，并参考[分词器概述](https://huggingface.co/transformers/tokenizer_summary.html)来了解子词分词算法之间的区别。

## Getting a corpus

我们需要文本来训练我们的分词器。我们将使用 [🤗 Datasets](https://github.com/huggingface/datasets) 库来下载我们的文本数据，可以通过 `load_dataset` 函数轻松实现：

In [3]:
from datasets import load_dataset

在这个例子中，我们将使用 Wikitext-2（其中包含 4.5MB 的文本，所以我们的示例训练会很快），但你可以使用任何你想要的数据集（以及任何语言，只要不是英语）。

In [4]:
dataset = load_dataset("wikitext", name="wikitext-2-raw-v1", split="train")

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

我们可以看一下数据集，它有36,718个文本:

In [5]:
dataset

Dataset({
    features: ['text'],
    num_rows: 36718
})

要访问一个元素，我们只需要提供它的索引:

In [6]:
dataset[1]

{'text': ' = Valkyria Chronicles III = \n'}

我们也可以直接访问切片，在这种情况下，我们得到一个键为“text”的字典和一个文本列表作为值:

In [7]:
dataset[:5]

{'text': ['',
  ' = Valkyria Chronicles III = \n',
  '',
  ' Senjō no Valkyria 3 : Unrecorded Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . Employing the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " Calamaty Raven " . \n',
  " The game began development in 2010 , carrying over a large portion of the work done on Valkyria Chronicles II . While it retained the standard features of the series , it also underwent multiple adjustments , such as making th

训练我们的tokenizer的API将需要文本批次的迭代器，例如文本列表的列表:

In [8]:
batch_size = 1000
all_texts = [dataset[i : i + batch_size]["text"] for i in range(0, len(dataset), batch_size)]

为了避免将所有内容加载到内存中(因为Datasets库将元素保存在磁盘上，只在请求时才将它们加载到内存中)，我们定义了一个Python迭代器。如果你有一个庞大的数据集，这是特别有用的:

In [9]:
def batch_iterator():
    for i in range(0, len(dataset), batch_size):
        yield dataset[i : i + batch_size]["text"]

现在让我们看看如何使用这个语料库来训练一个新的tokenizer!有两个api可以做到这一点:第一个使用现有的tokenizer，并将在一行代码中在语料库上训练它的新版本，第二个是实际构建您的tokenizer块，因此可以让您自定义每一步!

## Using an existing tokenizer

如果你想使用与现有分词器完全相同的算法和参数来训练一个新的分词器，你可以简单地使用 `train_new_from_iterator` API。例如，让我们使用相同的分词算法在 Wikitext-2 上训练一个新版本的 GPT-2 分词器。

首先，我们需要加载我们想要使用的分词器作为模型：

In [10]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

确保您选择的tokenizer为*快速*版本(由🤗Tokenizers库支持)，否则笔记本的其余部分将无法运行:

In [11]:
tokenizer.is_fast

True

然后我们将训练语料库(列表的列表或我们之前定义的迭代器)提供给`train_new_from_iterator`方法。我们还必须指定要使用的词汇表大小:

In [12]:
new_tokenizer = tokenizer.train_new_from_iterator(batch_iterator(), vocab_size=25000)

这就是它的全部!多亏了由Rust支持的🤗Tokenizers库，训练进行得非常快。

现在，您已经准备好了一个新的tokenizer来预处理数据并训练语言模型。你可以像往常一样给它输入文本:

In [13]:
new_tokenizer(dataset[:5]["text"])

{'input_ids': [[], [301, 8639, 9504, 3050, 301, 315], [], [4720, 74, 4825, 889, 8639, 491, 529, 672, 6944, 475, 267, 9504, 374, 2809, 529, 10879, 231, 100, 162, 255, 113, 14391, 4046, 113, 4509, 95, 18351, 4509, 256, 4046, 99, 4046, 22234, 96, 19, 264, 6437, 272, 8639, 281, 261, 3518, 2035, 491, 373, 264, 5162, 3305, 290, 344, 8639, 9504, 3050, 2616, 1822, 264, 364, 259, 14059, 1559, 340, 2393, 1527, 737, 1961, 370, 805, 3604, 288, 7577, 14, 54, 782, 337, 261, 4840, 15585, 272, 19958, 284, 1404, 1696, 284, 1822, 264, 385, 364, 261, 1431, 737, 284, 261, 8639, 906, 272, 2531, 1858, 286, 261, 1112, 9658, 281, 14059, 288, 1626, 340, 645, 6556, 344, 520, 14434, 264, 261, 1485, 3436, 7515, 290, 261, 518, 737, 288, 4750, 261, 302, 22039, 302, 264, 259, 21720, 1743, 3836, 5654, 261, 4259, 281, 4742, 490, 724, 261, 3581, 1351, 283, 1114, 579, 952, 4010, 1985, 2563, 288, 453, 2128, 807, 935, 261, 7655, 3836, 302, 2038, 314, 271, 89, 22414, 302, 272, 315], [324, 737, 1022, 1984, 284, 1525, 264, 7

你可以使用save_pretrained方法将其保存在本地:

In [14]:
new_tokenizer.save_pretrained("my-new-tokenizer")

('my-new-tokenizer/tokenizer_config.json',
 'my-new-tokenizer/special_tokens_map.json',
 'my-new-tokenizer/vocab.json',
 'my-new-tokenizer/merges.txt',
 'my-new-tokenizer/added_tokens.json',
 'my-new-tokenizer/tokenizer.json')

或者甚至把它推到[huggingface hub](https://huggingface.co/models)，在任何地方使用新的tokenizer。只需在终端中执行`huggingface-cli login`或执行以下单元格，确保您已经存储了身份验证令牌:

In [15]:
from huggingface_hub import notebook_login

notebook_login()

我们就快完成了，你还需要安装`git lfs`。您可以通过取消注释以下单元格直接从本笔记本中完成:

In [ ]:
# !apt install git-lfs

In [16]:
new_tokenizer.push_to_hub("my-new-shiny-tokenizer")

CommitInfo(commit_url='https://huggingface.co/weege007/my-new-shiny-tokenizer/commit/376c3f93d3eab1f18a03d5b65522056600092e6e', commit_message='Upload tokenizer', commit_description='', oid='376c3f93d3eab1f18a03d5b65522056600092e6e', pr_url=None, pr_revision=None, pr_num=None)

现在可以在这台机器上重新加载tokenizer

In [17]:
tok = new_tokenizer.from_pretrained("my-new-tokenizer")

或者从任何地方使用仓库 ID 进行加载，仓库 ID 是你的命名空间后跟一个斜杠，然后是你在 `push_to_hub` 方法中给出的名称，例如：

```python
tok = new_tokenizer.from_pretrained("weege007/my-new-shiny-tokenizer")
```

现在，如果您想创建和训练一个看起来不像现有任何东西的tokenizer，则需要使用🤗Tokenizers库从头开始构建它。

## Building your tokenizer from scratch

为了理解如何从头开始构建你的分词器，我们需要更深入地了解 🤗 Tokenizers 库和分词流程。这个流程包括几个步骤：

- **规范化（Normalization）**：对初始输入字符串执行所有初始转换。例如，当你需要将一些文本转换为小写，可能去除空格，甚至应用常见的 unicode 规范化过程时，你会添加一个规范化器（Normalizer）。
- **预分词化（Pre-tokenization）**：负责分割初始输入字符串。这是决定在何处以及如何预分割原始字符串的组件。最简单的例子是简单地按空格分割。
- **模型（Model）**：处理所有子词的发现和生成，这是可训练的部分，真正依赖于你的输入数据。
- **后处理（Post-Processing）**：提供高级构造功能，以与一些基于 Transformers 的 SOTA 模型兼容。例如，对于 BERT，它会将分词后的句子包装在 [CLS] 和 [SEP] 标记周围。

另外，还有一个反向的过程：

- **解码（Decoding）**：负责将分词后的输入映射回原始字符串。解码器通常根据之前使用的 `PreTokenizer` 来选择。

对于模型的训练，🤗 Tokenizers 库提供了一个 `Trainer` 类，我们将使用它。

所有这些构建模块都可以组合起来创建工作的分词流程。为了给你一些例子，我们将展示三个完整的流程：如何复制 GPT-2、BERT 和 T5（这将给你一个 BPE、WordPiece 和 Unigram 分词器的例子）。

### WordPiece model like BERT

让我们看看如何创建一个类似于用于训练 BERT 的 WordPiece 分词器。第一步是创建一个带有空的 `WordPiece` 模型的 `Tokenizer`：

In [18]:
from tokenizers import decoders, models, normalizers, pre_tokenizers, processors, trainers, Tokenizer

tokenizer = Tokenizer(models.WordPiece(unk_token="[UNK]"))

这个 `tokenizer` 还没有准备好进行训练。我们需要添加一些预处理步骤：规范化（可选）和预分词器，它将输入分割成我们称之为单词的块。然后，标记将成为这些单词的一部分（但不能比单词更大）。

对于 BERT，规范化是转换为小写。由于 BERT 是一个如此流行的模型，它有自己的规范化器：

In [19]:
tokenizer.normalizer = normalizers.BertNormalizer(lowercase=True)

如果你想自定义它，你可以使用现有的模块并将它们组合成一个序列：例如，在这里我们将转换为小写，应用 NFD 规范化并去除重音：

In [20]:
tokenizer.normalizer = normalizers.Sequence(
    [normalizers.NFD(), normalizers.Lowercase(), normalizers.StripAccents()]
)

还有一个 `BertPreTokenizer`，我们可以直接使用它。它使用空格和标点进行预分词：

In [21]:
tokenizer.pre_tokenizer = pre_tokenizers.BertPreTokenizer()

和规范化器一样，我们可以将多个预分词器组合在一个 `Sequence` 中。如果我们想快速查看它如何预处理输入，我们可以调用 `pre_tokenize_str` 方法：

In [22]:
tokenizer.pre_tokenizer.pre_tokenize_str("This is an example!")

[('This', (0, 4)),
 ('is', (5, 7)),
 ('an', (8, 10)),
 ('example', (11, 18)),
 ('!', (18, 19))]

请注意，预分词器不仅将文本分割为单词，还保留了偏移量，即每个单词在原始文本中的起始和结束位置。这将允许最终的分词器将每个标记与它来自的文本部分匹配（这是我们用于问答或标记分类任务的一个特性）。

现在我们可以训练我们的分词器了（流程还没有完全结束，但我们需要一个训练好的分词器来构建后处理器），我们使用 `WordPieceTrainer` 进行训练。要记住的关键是将特殊的标记传递给训练器，因为它们在语料库中看不到。

In [23]:
special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
trainer = trainers.WordPieceTrainer(vocab_size=25000, special_tokens=special_tokens)

要实际训练分词器，方法看起来与之前使用的类似：我们可以传递一些文本文件，或者是一批文本的迭代器：

In [24]:
tokenizer.train_from_iterator(batch_iterator(), trainer=trainer)

现在分词器已经训练好了，我们可以定义后处理器：我们需要在开头添加 CLS 标记，在结尾添加 SEP 标记（对于单个句子），或者添加多个 SEP 标记（对于句子对）。我们使用 [`TemplateProcessing`](https://huggingface.co/docs/tokenizers/python/latest/api/reference.html#tokenizers.processors.TemplateProcessing) 来完成这个任务，这需要知道 CLS 和 SEP 标记的 ID（这就是为什么我们等到训练完成才做的）。

所以让我们首先获取这两个特殊标记的 ID：

In [25]:
cls_token_id = tokenizer.token_to_id("[CLS]")
sep_token_id = tokenizer.token_to_id("[SEP]")
print(cls_token_id, sep_token_id)

2 3


以下是我们如何构建后处理器。我们必须在模板中指示如何使用一个句子 (`$A`) 或两个句子 (`$A` 和 `$B`) 来组织特殊的标记。冒号后面跟着一个数字，表示要给每个部分分配的标记类型 ID。

In [26]:
tokenizer.post_processor = processors.TemplateProcessing(
    single=f"[CLS]:0 $A:0 [SEP]:0",
    pair=f"[CLS]:0 $A:0 [SEP]:0 $B:1 [SEP]:1",
    special_tokens=[
        ("[CLS]", cls_token_id),
        ("[SEP]", sep_token_id),
    ],
)

我们可以通过编码一对句子来检查是否获得了预期的结果：

In [27]:
encoding = tokenizer.encode("This is one sentence.", "With this one we have a pair.")

我们可以查看标记来检查特殊标记是否已经插入到了正确的位置：

In [28]:
encoding.tokens

['[CLS]',
 'this',
 'is',
 'one',
 'sentence',
 '.',
 '[SEP]',
 'with',
 'this',
 'one',
 'we',
 'have',
 'a',
 'pair',
 '.',
 '[SEP]']

我们还可以检查标记类型 ID 是否正确：

In [29]:
encoding.type_ids

[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]

在这个分词器的最后一个部分是解码器，我们使用一个 `WordPiece` 解码器，并指示特殊的前缀 `##`：

In [30]:
tokenizer.decoder = decoders.WordPiece(prefix="##")

现在我们的分词器已经完成，我们需要将它包装在一个 Transformers 对象中，以便能够与 Transformers 库一起使用。更具体地说，我们必须将它放在与我们想要使用的模型相对应的分词器快速类中，这里是一个 `BertTokenizerFast`：

In [31]:
from transformers import BertTokenizerFast

new_tokenizer = BertTokenizerFast(tokenizer_object=tokenizer)

和之前一样，我们可以将这个分词器作为普通的 Transformers 分词器使用，并使用 `save_pretrained` 或 `push_to_hub` 方法。

如果你正在构建的分词器与 Transformers 中的任何类不匹配，因为它确实很特殊，你可以将它包装在 `PreTrainedTokenizerFast` 中。

### BPE model like GPT-2

现在让我们看看如何创建一个类似于用于训练 GPT-2 的 BPE 分词器。第一步是创建一个带有空的 `BPE` 模型的 `Tokenizer`：

In [32]:
tokenizer = Tokenizer(models.BPE())

和之前一样，我们必须添加可选的规范化步骤（在 GPT-2 的情况下不使用），并且在训练之前需要指定一个预分词器。在 GPT-2 的情况下，使用的预分词器是字节级别的预分词器：

In [33]:
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)

如果我们想快速查看它如何预处理输入，我们可以调用 `pre_tokenize_str` 方法：

In [34]:
tokenizer.pre_tokenizer.pre_tokenize_str("This is an example!")

[('This', (0, 4)),
 ('Ġis', (4, 7)),
 ('Ġan', (7, 10)),
 ('Ġexample', (10, 18)),
 ('!', (18, 19))]

我们使用了与 GPT-2 相同的默认前缀空格，所以你可以看到每个单词在开头都加了一个 `'Ġ'`，除了第一个单词。

现在我们可以训练我们的分词器了！这次我们使用的是 `BpeTrainer`。

In [35]:
trainer = trainers.BpeTrainer(vocab_size=25000, special_tokens=["<|endoftext|>"])
tokenizer.train_from_iterator(batch_iterator(), trainer=trainer)

为了完成整个流程，我们需要包括后处理器和解码器：

In [36]:
tokenizer.post_processor = processors.ByteLevel(trim_offsets=False)
tokenizer.decoder = decoders.ByteLevel()

和之前一样，我们最后将其包装在一个 Transformers 分词器对象中：

In [37]:
from transformers import GPT2TokenizerFast

new_tokenizer = GPT2TokenizerFast(tokenizer_object=tokenizer)

### Unigram model like Albert

现在让我们看看如何创建一个类似于用于训练 T5 的 Unigram 分词器。第一步是创建一个带有空的 `Unigram` 模型的 `Tokenizer`：

In [38]:
tokenizer = Tokenizer(models.Unigram())

和之前一样，我们需要添加可选的规范化步骤（这里是一些替换和转换为小写），并且在训练之前需要指定一个预分词器。所使用的预分词器是一个 `Metaspace` 预分词器：它将所有空格替换为一个特殊字符（默认为 ▁），然后在该字符上分割。

In [39]:
tokenizer.normalizer = normalizers.Sequence(
    [normalizers.Replace("``", '"'), normalizers.Replace("''", '"'), normalizers.Lowercase()]
)
tokenizer.pre_tokenizer = pre_tokenizers.Metaspace()

如果我们想快速查看它如何预处理输入，我们可以调用 `pre_tokenize_str` 方法：

In [40]:
tokenizer.pre_tokenizer.pre_tokenize_str("This is an example!")

[('▁This', (0, 4)), ('▁is', (4, 7)), ('▁an', (7, 10)), ('▁example!', (10, 19))]

你可以看到每个单词在开头都加了一个初始的 `▁`，这通常是由 sentencepiece 完成的。

现在我们可以训练我们的分词器了！这次我们使用的是 `UnigramTrainer`。我们必须在这个训练器中明确设置未知标记，否则它会在训练之后遗忘它。

In [41]:
trainer = trainers.UnigramTrainer(vocab_size=25000, special_tokens=["[CLS]", "[SEP]", "<unk>", "<pad>", "[MASK]"], unk_token="<unk>")
tokenizer.train_from_iterator(batch_iterator(), trainer=trainer)

为了完成整个流程，我们需要包括后处理器和解码器。后处理器与我们在 BERT 中看到的非常相似，解码器只是 `Metaspace`，就像预分词器中一样。

In [42]:
cls_token_id = tokenizer.token_to_id("[CLS]")
sep_token_id = tokenizer.token_to_id("[SEP]")

In [43]:
tokenizer.post_processor = processors.TemplateProcessing(
    single="[CLS]:0 $A:0 [SEP]:0",
    pair="[CLS]:0 $A:0 [SEP]:0 $B:1 [SEP]:1",
    special_tokens=[
        ("[CLS]", cls_token_id),
        ("[SEP]", sep_token_id),
    ],
)
tokenizer.decoder = decoders.Metaspace()

和之前一样，我们最后将其包装在一个 Transformers 分词器对象中：

In [45]:
from transformers import AlbertTokenizerFast

new_tokenizer = AlbertTokenizerFast(tokenizer_object=tokenizer)
print(new_tokenizer)

AlbertTokenizerFast(name_or_path='', vocab_size=25000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '[CLS]', 'eos_token': '[SEP]', 'unk_token': '<unk>', 'sep_token': '[SEP]', 'pad_token': '<pad>', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("[MASK]", rstrip=False, lstrip=True, single_word=False, normalized=False, special=True),
}


## Use your new tokenizer to train a language model!

你可以在语言建模从头开始的笔记本中使用你的新分词器，或者在[语言建模脚本](https://github.com/huggingface/transformers/tree/master/examples/pytorch/language-modeling)中使用 `--tokenizer_name` 参数来使用它，从头开始训练一个模型。